In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,from_json
import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [9]:
def kafka_parameters(kafka_bootstrap_servers,kafka_topic):
    # Configuración de lectura desde Kafka
    kafka_bootstrap_servers = "kafka:9092"  # Cambia al host y puerto de tu cluster Kafka
    kafka_topic = "test"  # Nombre del tópico de Kafka
    
    kafkaParams = {
        "kafka.bootstrap.servers": f"{kafka_bootstrap_servers}",  # Replace with your broker addresses
        "subscribe":f"{kafka_topic}",  # Replace with your topic name
        "startingOffsets": "earliest",  # or "earliest"
        "key.deserializer": "org.apache.kafka.common.serialization.StringDeserializer",
        "value.deserializer": "org.apache.kafka.common.serialization.StringDeserializer"
    }
        
    return kafkaParams


In [16]:
def read_stream(parameters):
    # Leer datos de Kafka
    kafka_df = spark.readStream \
        .format("kafka") \
        .options(**parameters)\
        .load()
    messages_df = kafka_df.select(
    col("key").cast("string").alias("key"),(col("value").cast("string").alias("value")))

    
    def print_messages(batch_df, batch_id):
        print(f"--- Batch ID: {batch_id} ---")
        for row in batch_df.collect():  # Recorrer las filas del microbatch
            print(f"Key: {row['key']}, Value: {row['value']}")
            
    #messages_df.select(from_json(col('value'), schema).alias("data")).select(("data.*"))
    # Imprimir los mensajes usando foreachBatch
    
    query = messages_df.writeStream \
        .foreachBatch(print_messages) \
        .option('checkpointLocation', './spark_kafka_checkpoint')\
        .outputMode("update") \
        .start()
        #.trigger(processingTime="2 seconds") \
    
    query.awaitTermination()

In [17]:
def wait_events(df_streaming):
    while True:
        status = df_streaming.status['message']
        
        print(status)
        
        if status == 'Stopped':
            df_streaming.stop()

        
        time.sleep(2)
        
    print('has been stop the transmition')

In [18]:
if __name__ == '__main__':
    spark = SparkSession.builder \
        .master("spark://bc74c068484d:7077") \
        .appName("KafkaSparkStreamingExample") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.4") \
        .getOrCreate()
    
    schema = StructType([
                StructField("name", StringType()),
                StructField("location", StringType()),
                StructField("phone", StringType()),
            ])

    
    
    read_stream(kafka_parameters("kafka:9092","test"))
    


    

--- Batch ID: 0 ---
Key: data, Value: {"name": "Michelle Leach", "location": "Richardton", "phone": "001-361-665-4965x5296"}
Key: data, Value: {"name": "Mikayla Benton", "location": "New Ryan", "phone": "+1-604-925-6950x63103"}
Key: data, Value: {"name": "Lauren Williams", "location": "Rodriguezburgh", "phone": "241-333-3152"}
Key: data, Value: {"name": "John Ramirez", "location": "South Markstad", "phone": "001-930-453-0230x087"}
Key: data, Value: {"name": "Brittney Mendoza", "location": "New Theresa", "phone": "791-267-2901"}
Key: data, Value: {"name": "Thomas Edwards", "location": "Vasquezfurt", "phone": "+1-511-234-5096x732"}
Key: data, Value: {"name": "Lisa Johnson", "location": "Andersonside", "phone": "001-212-549-0988"}
Key: data, Value: {"name": "Kenneth Cortez", "location": "South Christopherbury", "phone": "353-815-7160x17813"}
Key: data, Value: {"name": "Cynthia Daniel", "location": "Kaitlinville", "phone": "785.258.7216x516"}
Key: data, Value: {"name": "Andrea Carpenter", 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 